In [1]:
require 'experiments'

experiment = basicGoExperiment:new { 
    name = "basicGoExperiment",
    dataset = ToyDataset,
    group = "toy",
    useCuda = false,
    validationSize = 200,
    iterations = 100,
    numLayers = 3,
    channelSize = 64,

    batchSize = 32,
    kernels = {5},
    strides = {1},
    channels = {37},

    
    rate = .01,
    rateDecay = 1e-7,

    criterion = nn.ClassNLLCriterion()
}

In [2]:
experiment:init()

In [3]:
experiment:run(30)

30	
